# Document Grounding on S3 data using SAP Cloud SDK for AI (Python)

Purpose: Ground LLM responses on your enterprise data stored in S3, with SAP Document Grounding service using Ochestration module of SAP Cloud SDK (Python). The tutorial demonstrates different steps to set up and implement the grounding service with S3 as a data source.


The process consists of three steps:  
* Step 1: Upload Documents to S3 bucket  
* Step 2: Create Data Repository pointing to S3 bucket  
* Step 3: Retrieve most similar documents from Data Repository based on input query and generate augmented answer


**Pre-requisites:**
* Object Store (S3)
* Orchestration service is deployed on AI Core. [SAP Help](https://help.sap.com/docs/ai-launchpad/sap-ai-launchpad/create-deployment-for-orchestration?q=orchestration)


**Step 1:**
Push your document(s) to the S3 bucket. 

**Step 2:**
Create Generic Secret and a data repository using AI Launchpad with S3 as a source.

**Step 3:**
Once a data repository is created from S3, use the SDK to retrieve similar documents and complete answer generation.


In this tutorial, we will:
1. Initialize the Orchestration Service and configure the LLM.
2. Define a prompt template for combining the user query with retrieved context.
3. Configure grounding with vector data repositories.
4. Run the orchestration pipeline and get the grounded response.

## Step 1: Data Load to S3

**Obtain Object Store Credentials**  
* Download S3 credentials from BTP cockpit > BTP subaccount > Space > Instances > ObjectStore > Credentials

**AWS CLI Installation and Configuration**  
* Install AWS CLI: Download and install the AWS Command Line Interface from the official AWS documentation [Link](https://docs.aws.amazon.com/cli/latest/userguide/getting-started-install.html).
* Verify Installation: Check your installation by running:
``` sh
aws --version
```
* Configure AWS CLI: Run the configuration command and pass corresponding values from Object Store credentials.
``` sh
aws configure
```

**Push Documents to S3**  
You can push your documents to S3 bucket. You can optionally put the documents to a sub-directory in the bucket as well.

S3 CLI commands:  
``` sh
aws s3 cp sample.pdf s3://your-bucket-name/sample.pdf
OR
aws s3 cp . s3://your-bucket-name/ --recursive
OR
aws s3 cp . s3://your-bucket-name/<optional_path>/ --recursive
```

## Step 2: Create Data Repository on AI Launchpad

Create Generic Secret key collection and a Data Repository using that on AI Launchpad. [SAP Help Documentation](https://help.sap.com/docs/ai-launchpad/sap-ai-launchpad/grounding-management).



## Step 3: Retrieve Documents from Data Repository

### Step 3.1: Initialize Orchestration Service and LLM

First, we connect to the **Orchestration Service** using the service URL of your deployment.
We also define the **LLM** (in this case `gpt-4o`) with required parameters such as temperature.

Replace the placeholder `<your_orchestration_deployment_url>` in the URL with your deployment ID.

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv(override=True)

In [ ]:
from gen_ai_hub.orchestration.service import OrchestrationService
from gen_ai_hub.orchestration.models.config import OrchestrationConfig
from gen_ai_hub.orchestration.models.document_grounding import (GroundingModule, GroundingType, DataRepositoryType,
                                                                GroundingFilterSearch, DocumentGrounding,DocumentGroundingFilter)
from gen_ai_hub.orchestration.models.llm import LLM
orchestration_service_url = "<your_orchestration_deployment_url>"
orchestration_service = OrchestrationService(api_url=orchestration_service_url)

llm = LLM(
    name="gpt-4o",
    parameters={
        'temperature': 0.0,
    }
)

### Step 3.2: Define Prompt Template

Here, we define a **prompt template** that instructs the model to:
- Take the retrieved `Context` from the document grounding service.
- Use it along with the user’s `Question`.

This ensures the LLM’s answers are **grounded in enterprise data** rather than generic knowledge.

In [ ]:
from gen_ai_hub.orchestration.models.message import SystemMessage, UserMessage
from gen_ai_hub.orchestration.models.template import Template, TemplateValue

prompt = Template(messages=[
        SystemMessage("You are a helpful assistant. Use the given Context information for providing answer to a given query."),
        UserMessage("""Context: {{ ?grounding_response }}
                       Question: {{ ?query }}
                    """),
])

### Step 3.3: Configure Document Grounding

Next, we configure **document grounding** by linking a **vector data repository**. This ensures that relevant enterprise documents are searched and retrieved for the query.

- `data_repositories`: List of repository IDs.
- `max_chunk_count`: Controls how many chunks of relevant documents are retrieved.

Replace the placeholder (`<repository_id>`) with your actual repository ID.

In [ ]:
filters = [DocumentGroundingFilter(id="vector",
                                   data_repositories=["<repository_id>"],
                                   search_config=GroundingFilterSearch(max_chunk_count=3),
                                   data_repository_type=DataRepositoryType.VECTOR.value
                                   )]

grounding_config = GroundingModule(
            type=GroundingType.DOCUMENT_GROUNDING_SERVICE.value,
            config=DocumentGrounding(input_params=["query"], output_param="grounding_response", filters=filters)
                   )

config = OrchestrationConfig(template=prompt, llm=llm, grounding=grounding_config)


### Step 3.4: Run the Orchestration Pipeline

Finally, we execute the orchestration pipeline by passing the query. The LLM combines the query with **retrieved context from documents** and returns a **grounded answer**.

You should see the response printed below once you run the last cell.

In [ ]:
from functools import partial

def _run_query(query: str, *, _service=orchestration_service, _config=config):
    response = _service.run(
        config=_config,
        template_values=[TemplateValue("query", query)]
    )
    print(response.orchestration_result.choices[0].message.content)

# Use partial so you can call it simply with a query
run_query = partial(_run_query)

In [9]:
run_query("what is effective receptive field?")

The effective receptive field is a concept introduced to better understand how deep convolutional neural networks (CNNs) process input images. While the theoretical receptive field of a unit in a CNN is determined by the network's architecture and is supposed to cover a certain area of the input image, the effective receptive field is the actual area that significantly influences the unit's output. 

Empirical studies have shown that the effective receptive field is smaller than the theoretical receptive field and follows a Gaussian distribution. This means that the central part of the theoretical receptive field has a stronger influence on the unit's output, while the outer parts contribute less. Understanding the effective receptive field is important for tasks like semantic image segmentation, stereo, and optical flow estimation, where each output pixel needs to respond to a sufficiently large area of the input image to capture all relevant information.

The study of effective recep

In [10]:
run_query("what is Document AI Embedded?")

The SAP Document AI Embedded Edition refers to a version of the SAP Document AI solution that is integrated within a specific workspace or environment. This edition allows users to utilize the capabilities of SAP Document AI directly within their existing systems or workflows, providing seamless access to document information extraction processes. It enables the automation of extracting relevant information from business documents and matching them to enrichment data records, all within the embedded workspace.
